In [50]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch_geometric.nn import GATConv
import pandas as pd
import numpy as np
import string

import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import re

In [51]:
!pip install tensorflow

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

In [80]:
#preprocess




df = pd.read_csv('Large_dataset1.csv', encoding = 'latin')

df

df = df.dropna()



df

def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text

df['Clean_Resumes'] = np.vectorize(remove_pattern)(df['text'], '@[\w]*')

df

clean_Resumes =[]

for index, row in df.iterrows():
    words_without_links = [word for word in row.Clean_Resumes.split() if 'http' not in word]
    clean_Resumes.append(' '.join(words_without_links)) 

df['Clean_Resumes'] = clean_Resumes
df.head(10)

df = df[df['Clean_Resumes']!= ''] #removes empty string

df

df.drop_duplicates(subset=['Clean_Resumes'],keep = False) # dropping the duplicate rows

def clean_text(text):
    text = text.lower()
    text = re.sub('!','', text)
    text = re.sub('\[.*?\]','', text)
    text = re.sub('➢','',text)
    text = re.sub('•','',text)
    text = re.sub('●','', text)
    text = re.sub('⚫', '', text)
    text = re.sub('https?://\S+|www.\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df.Clean_Resumes = df.Clean_Resumes.apply(lambda x: clean_text(x))
df.head()

my_stop_words = stopwords.words('english')

cleaned_resumes = []

for index, row in df.iterrows():
    words_without_stopwords = [word for word in row.Clean_Resumes.split() if word not in my_stop_words]
    cleaned_resumes.append(' '.join(words_without_stopwords))

df['Absolute_Clean_Resumes'] = cleaned_resumes
df.head(10)

Tokenized_Resume = df['Absolute_Clean_Resumes'].apply(lambda x: x.split())
Tokenized_Resume.head(10)

word_lemmatizer = WordNetLemmatizer()

Tokenized_Resume = Tokenized_Resume.apply(lambda x: [word_lemmatizer.lemmatize(i) for i in x])
Tokenized_Resume.head(10)

pos_tags = []
for resume in Tokenized_Resume:
    pos_tags.append(pos_tag(resume))

filtered_tokens = []
for tags in pos_tags:
    tokens = [word for word, tag in tags if tag in ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'RB', 'RBR', 'RBS']]
    filtered_tokens.append(' '.join(tokens))

df['Absolute_Clean_Resumes'] = filtered_tokens
df

,text,labels,job_description,Clean_Resumes,Absolute_Clean_Resumes
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,hr administratormarketing associate hr adminis...,hr administratormarketing associate hr adminis...
1,"HR SPECIALIST, US HR OPERATIONS ...",Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,hr specialist us hr operations summary versati...,hr specialist u hr operation summary versatile...
2,HR DIRECTOR Summary Over 2...,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,hr director summary over years experience in ...,hr director summary year experience recruiting...
3,HR SPECIALIST Summary Dedica...,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,hr specialist summary dedicated driven and dyn...,hr specialist summary dedicated driven dynamic...
4,HR MANAGER Skill Highlights ...,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,hr manager skill highlights hr skills hr depar...,hr manager skill highlight hr skill hr departm...
...,...,...,...,...,...
3439,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,BPO / Call Centre / ITES,Job Description Send me Jobs like this We wi...,rank non commissioned officer in charge briga...,rank non commissioned officer charge brigade m...
3440,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",IT-Software / Software Services,Job Description Send me Jobs like this Provi...,government relations communications and organi...,government relation communication organization...
3441,GEEK SQUAD AGENT Professional...,Medical / Healthcare / Hospitals,Job Description Send me Jobs like this Job D...,geek squad agent professional profile it suppo...,geek squad agent professional profile support ...
3442,PROGRAM DIRECTOR / OFFICE MANAGER ...,IT-Software / Software Services,Job Description Send me Jobs like this Greet...,program director office manager summary highl...,program director office manager summary highly...


In [85]:
df['Clean_job_description'] = np.vectorize(remove_pattern)(df['job_description'], '@[\w]*')

Clean_job_description =[]

for index, row in df.iterrows():
    words_without_links = [word for word in row.Clean_job_description.split() if 'http' not in word]
    Clean_job_description.append(' '.join(words_without_links)) 

df['Clean_job_description'] = Clean_job_description

df = df[df['Clean_job_description']!= '']

df

df.Clean_job_description = df.Clean_job_description.apply(lambda x: clean_text(x))
df.head()

Clean_job_description = []
for index, row in df.iterrows():
    words_without_stopwords = [word for word in row.Clean_job_description.split() if word not in my_stop_words]
    Clean_job_description.append(' '.join(words_without_stopwords))

df['Clean_job_description'] = Clean_job_description
df.head(10)

Tokenized_JD = df['Clean_job_description'].apply(lambda x: x.split())
Tokenized_JD.head(10)

Tokenized_JD = Tokenized_JD.apply(lambda x: [word_lemmatizer.lemmatize(i) for i in x])

df['Clean_job_description'] = Tokenized_JD

clean_jd =[]
for row in df['Clean_job_description']:
    joining = ' '.join(row)
    clean_jd.append(joining)
    
df['Clean_job_description'] = clean_jd

df


,text,labels,job_description,Clean_Resumes,Absolute_Clean_Resumes,Clean_job_description
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,hr administratormarketing associate hr adminis...,hr administratormarketing associate hr adminis...,job description send job like qualification gr...
1,"HR SPECIALIST, US HR OPERATIONS ...",Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,hr specialist us hr operations summary versati...,hr specialist u hr operation summary versatile...,job description send job like qualification gr...
2,HR DIRECTOR Summary Over 2...,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,hr director summary over years experience in ...,hr director summary year experience recruiting...,job description send job like developer provid...
3,HR SPECIALIST Summary Dedica...,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,hr specialist summary dedicated driven and dyn...,hr specialist summary dedicated driven dynamic...,job description send job like involved stage i...
4,HR MANAGER Skill Highlights ...,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,hr manager skill highlights hr skills hr depar...,hr manager skill highlight hr skill hr departm...,job description send job like please share res...
...,...,...,...,...,...,...
3439,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,BPO / Call Centre / ITES,Job Description Send me Jobs like this We wi...,rank non commissioned officer in charge briga...,rank non commissioned officer charge brigade m...,job description send job like hire mentioned s...
3440,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",IT-Software / Software Services,Job Description Send me Jobs like this Provi...,government relations communications and organi...,government relation communication organization...,job description send job like provide support ...
3441,GEEK SQUAD AGENT Professional...,Medical / Healthcare / Hospitals,Job Description Send me Jobs like this Job D...,geek squad agent professional profile it suppo...,geek squad agent professional profile support ...,job description send job like job description ...
3442,PROGRAM DIRECTOR / OFFICE MANAGER ...,IT-Software / Software Services,Job Description Send me Jobs like this Greet...,program director office manager summary highl...,program director office manager summary highly...,job description send job like greeting truetec...


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2713 entries, 0 to 3443
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    2713 non-null   object
 1   labels                  2713 non-null   object
 2   job_description         2713 non-null   object
 3   Clean_Resumes           2713 non-null   object
 4   Absolute_Clean_Resumes  2713 non-null   object
 5   Clean_job_description   2713 non-null   object
dtypes: object(6)
memory usage: 148.4+ KB


In [87]:
resumes = df['Absolute_Clean_Resumes'].tolist()

job_descriptions = df['Clean_job_description'].tolist()

labels = df['labels'].tolist()

In [88]:
labels

['Media / Entertainment / Internet',
 'Advertising / PR / MR / Event Management',
 'IT-Software / Software Services',
 'Banking / Financial Services / Broking',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'Aviation / Aerospace Firms',
 'Industrial Products / Heavy Machinery',
 'IT-Software / Software Services',
 'FMCG / Foods / Beverage',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'Recruitment / Staffing',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'Recruitment / Staffing',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'Industrial Products / Heavy Machinery',
 'IT-Software / Software Services',
 'IT-Software / Software Services',
 'Banking / Financial Services / Broking',
 'Banking / Financial Services / Broking',
 'Internet / Eco

In [89]:
word_to_vec = {}
with open(r'C:\Users\sathv\OneDrive\Desktop\Resume Classification\glove.6B.300d.txt', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        word_to_vec[word] = vector

In [90]:
max(len(data) for data in resumes)

24243

In [91]:
max(len(data) for data in job_descriptions)

12135

In [92]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(job_descriptions)

word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

20280

In [93]:
sequences = tokenizer.texts_to_sequences(job_descriptions)
padded_seq = pad_sequences(sequences, maxlen=300, padding = 'post', truncating = 'post')

In [94]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(resumes)

word_index = tokenizer.word_index
vocab_size1 = len(word_index)
vocab_size1

20280

In [95]:
sequences1 = tokenizer1.texts_to_sequences(resumes)
padded_seq1 = pad_sequences(sequences, maxlen=300, padding = 'post', truncating = 'post')

In [96]:
padded_seq1.shape

(2713, 300)

In [98]:
padded_seq.shape

(2713, 300)

job_desc_features = []
resume_features = []

for description in job_descriptions:
    tokens = word_tokenize(description)
    embedding_matrix = [word_to_vec[token] for token in tokens if token in word_to_vec]
    if embedding_matrix:
        mean_embedding = np.mean(embedding_matrix, axis=0)
        job_desc_features.append(mean_embedding)

for resume in resumes:
    tokens = word_tokenize(resume)
    embedding_matrix = [word_to_vec[token] for token in tokens if token in word_to_vec]
    if embedding_matrix:
        mean_embedding = np.mean(embedding_matrix, axis=0)
        resume_features.append(mean_embedding)

job_desc_features = np.array(job_desc_features)
resume_features = np.array(resume_features)

job_desc_features.shape

resume_features.shape

In [100]:
embedding_matrix = np.zeros((vocab_size+1, 300))
for word, i in word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [101]:
resume_embeddings = embedding_matrix[padded_seq]
job_description_embeddings = embedding_matrix[padded_seq1]

In [102]:
resume_embeddings.shape

(2713, 300, 300)

In [104]:
job_description_embeddings.shape

(2713, 300, 300)

In [105]:
resume_embeddings = resume_embeddings.reshape(resume_embeddings.shape[0], -1)
job_description_embeddings = job_description_embeddings.reshape(job_description_embeddings.shape[0], -1)

In [106]:
resume_embeddings.shape

(2713, 90000)

In [107]:
adj = cosine_similarity(resume_embeddings, job_description_embeddings)

In [65]:
adj.shape

(2713, 2698)

In [62]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [63]:
encoded_labels

array([30,  1, 23, ..., 31, 23, 10], dtype=int64)

In [15]:
labels = torch.tensor(encoded_labels)

In [16]:
labels

tensor([0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2])

In [17]:
X = torch.tensor(job_desc_features, dtype=torch.float32)
y = torch.tensor(labels)

C:\Users\sathv\AppData\Local\Temp\ipykernel_9456\761218636.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(labels)


In [18]:
adj = torch.tensor(adj, dtype=torch.float32)

In [19]:
import torch.nn.functional as F

In [39]:

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.attention_layer1 = nn.Linear(in_features, out_features)
        self.attention_layer2 = nn.Linear(in_features, out_features)

    def forward(self, X, adj):
        h1 = F.leaky_relu(self.attention_layer1(X), negative_slope=0.2)
        h2 = F.leaky_relu(self.attention_layer2(X), negative_slope=0.2)

        N = h1.size()[0]
        h1_repeated = h1.repeat(1, N, 1).view(N, N, -1)
        h1_broadcasted = h1.repeat(N, 1, 1)
        a1_input = torch.cat([h1_repeated, h1_broadcasted], dim=2)
        e1 = torch.matmul(a1_input, self.a).squeeze(2)
        e1 = torch.where(adj > 0, e1, torch.tensor(-9e15).to(device))  # Masked attention

        attention1 = F.softmax(e1, dim=1)
        attention1 = F.dropout(attention1, self.dropout, training=self.training)
        h1_prime = torch.matmul(attention1, h1)

        h2_repeated = h2.repeat(1, N, 1).view(N, N, -1)
        h2_broadcasted = h2.repeat(N, 1, 1)
        a2_input = torch.cat([h2_repeated, h2_broadcasted], dim=2)
        e2 = torch.matmul(a2_input, self.a).squeeze(2)
        e2 = torch.where(adj > 0, e2, torch.tensor(-9e15).to(device))  # Masked attention

        attention2 = F.softmax(e2, dim=1)
        attention2 = F.dropout(attention2, self.dropout, training=self.training)
        h2_prime = torch.matmul(attention2, h2)

        # Concatenate attention heads
        h_prime = torch.cat([h1_prime, h2_prime], dim=1)
        h_prime = h_prime.view(N, -1)

        return h_prime


class GATClassifier(nn.Module):
    def __init__(self, in_features, num_classes, num_heads, dropout):
        super(GATClassifier, self).__init__()
        self.num_heads = num_heads

        self.attention_heads = nn.ModuleList()
        for _ in range(num_heads):
            self.attention_heads.append(GraphAttentionLayer(in_features, in_features, dropout, alpha=0.2))

        self.out_layer = nn.Linear(in_features * num_heads * 2, num_classes)

    def forward(self, X, adj):
        head_outputs = []
        for attn_head in self.attention_heads:
            head_output = attn_head(X, adj)
            head_outputs.append(head_output)

        h_cat = torch.cat(head_outputs, dim=1)
        output = self.out_layer(h_cat)

        return output


import torch.nn.functional as F

# Define the GAT model
class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.attention_layer1 = nn.Linear(in_features, out_features)
        self.attention_layer2 = nn.Linear(in_features, out_features)

    def forward(self, X, adj):
        h1 = F.leaky_relu(self.attention_layer1(X), negative_slope=0.2)
        h2 = F.leaky_relu(self.attention_layer2(X), negative_slope=0.2)

        N = h1.size()[0]
        h1_repeated = h1.unsqueeze(1).repeat(1, N, 1)
        h1_broadcasted = h1.unsqueeze(0).repeat(N, 1, 1)
        a1_input = torch.cat([h1_repeated, h1_broadcasted], dim=2)
        e1 = torch.matmul(a1_input, self.a).squeeze(2)
        e1 = torch.where(adj > 0, e1, torch.tensor(-9e15).to(device))  # Masked attention

        attention1 = F.softmax(e1, dim=1)
        attention1 = F.dropout(attention1, self.dropout, training=self.training)
        h1_prime = torch.matmul(attention1, h1)

        h2_repeated = h2.unsqueeze(1).repeat(1, N, 1)
        h2_broadcasted = h2.unsqueeze(0).repeat(N, 1, 1)
        a2_input = torch.cat([h2_repeated, h2_broadcasted], dim=2)
        e2 = torch.matmul(a2_input, self.a).squeeze(2)
        e2 = torch.where(adj > 0, e2, torch.tensor(-9e15).to(device))  # Masked attention

        attention2 = F.softmax(e2, dim=1)
        attention2 = F.dropout(attention2, self.dropout, training=self.training)
        h2_prime = torch.matmul(attention2, h2)

        # Apply average pooling and dropout to the node representation
        h1_prime = torch.mean(h1_prime, dim=1)
        h2_prime = torch.mean(h2_prime, dim=1)
        h_prime = torch.cat([h1_prime, h2_prime], dim=1)
        h_prime = F.dropout(h_prime, self.dropout, training=self.training)
        h_prime = h_prime.view(N, -1)

        return h_prime


class GATClassifier(nn.Module):
    def __init__(self, in_features, num_classes, num_heads, dropout):
        super(GATClassifier, self).__init__()
        self.num_heads = num_heads

        self.attention_heads = nn.ModuleList()
        for _ in range(num_heads):
            self.attention_heads.append(GraphAttentionLayer(in_features, in_features, dropout, alpha=0.2))

        self.out_layer = nn.Linear(in_features * num_heads * 2, num_classes)

    def forward(self, X, adj):
        head_outputs = []
        for attn_head in self.attention_heads:
            head_output = attn_head(X, adj)
            head_outputs.append(head_output)

        h_cat = torch.cat(head_outputs, dim=1)
        output = self.out_layer(h_cat)

        return output


In [40]:
num_features = X.shape[1]
num_classes = len(set(labels))
num_heads = 8
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [41]:
gat_model = GATClassifier(num_features, num_classes, num_heads, dropout).to(device)

In [42]:
X.shape[1]

300

In [43]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gat_model.parameters(), lr=0.01)

In [44]:
X_dense = X.to_dense()
adj_dense = adj.to_dense()

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
# Train the GAT model
num_epochs = 100
total_accuracy = 0
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = gat_model(X_dense, adj_dense)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    # Convert the output probabilities to class predictions
    _, predicted_labels = torch.max(output, 1)
    predicted_labels = predicted_labels.cpu().numpy()
    true_labels = y.cpu().numpy()

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    total_accuracy += accuracy
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/100, Loss: 3.4061, Accuracy: 0.0000
Epoch 2/100, Loss: 1.6299, Accuracy: 0.2000
Epoch 3/100, Loss: 4.6702, Accuracy: 0.2000
Epoch 4/100, Loss: 4.8257, Accuracy: 0.2000
Epoch 5/100, Loss: 5.8994, Accuracy: 0.2000
Epoch 6/100, Loss: 5.8824, Accuracy: 0.2000
Epoch 7/100, Loss: 2.6350, Accuracy: 0.2000
Epoch 8/100, Loss: 3.0726, Accuracy: 0.2333
Epoch 9/100, Loss: 2.8608, Accuracy: 0.2000
Epoch 10/100, Loss: 2.6740, Accuracy: 0.2000
Epoch 11/100, Loss: 2.6519, Accuracy: 0.2000
Epoch 12/100, Loss: 2.1927, Accuracy: 0.2000
Epoch 13/100, Loss: 2.1082, Accuracy: 0.2000
Epoch 14/100, Loss: 2.1895, Accuracy: 0.2000
Epoch 15/100, Loss: 2.0608, Accuracy: 0.2000
Epoch 16/100, Loss: 1.7022, Accuracy: 0.2000
Epoch 17/100, Loss: 1.9999, Accuracy: 0.2000
Epoch 18/100, Loss: 1.8897, Accuracy: 0.1333
Epoch 19/100, Loss: 1.6855, Accuracy: 0.2333
Epoch 20/100, Loss: 1.8392, Accuracy: 0.2000
Epoch 21/100, Loss: 1.8628, Accuracy: 0.2000
Epoch 22/100, Loss: 1.6437, Accuracy: 0.2333
Epoch 23/100, Loss:

In [47]:
average_accuracy = total_accuracy / num_epochs
print("Average Accuracy:", average_accuracy)

Average Accuracy: 0.1879999999999999


In [48]:
predicted_labels

array([3, 3, 3, 2, 3, 1, 3, 2, 2, 3, 3, 4, 3, 3, 3, 2, 3, 4, 2, 2, 4, 3,
       3, 3, 3, 3, 3, 3, 1, 3], dtype=int64)

In [49]:
true_labels

array([0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2], dtype=int64)